In [9]:
import pandas as pd
import numpy as np

# Constructing the dataframe

In [ ]:
all_tracks=pd.read_csv("../../data/tracks.csv")

In [11]:
labels=all_tracks.values
label_genre_top=np.where(labels[0,:]=="genre_top")
label_genres=np.where(labels[0,:]=="genres")
label_genres_all=np.where(labels[0,:]=="genres_all")
label_subset=np.where(labels[0,:]=="subset")
label_id=np.where(labels[0,:]=="id")
label_bit_rate=np.where(labels[0,:]=="bit_rate")


all_id=labels[2:, 0]
all_label_subset=labels[2:,label_subset[0][0]]
all_genre_top=labels[2:,label_genre_top[0][0]]
all_genres=labels[2:,label_genres[0][0]]
all_genres_all=labels[2:,label_genres_all[0][0]]
all_bit_rate=labels[2:,label_bit_rate[0][0]]

In [ ]:
print("all_genre_top", set(all_genre_top))

In [13]:
number=6
path_array=[]
folder_path="..\\..\\data\\fma_large"
for id in all_id:
    id_len=len(str(id))
    formatted_id=str(id)
    if id_len!=number:
        formatted_id=str(id).zfill(number)
    st=formatted_id[0:3:1]
    sub_folder_path="\\"+st+"\\"+formatted_id+".mp3"
    mp3_path=folder_path+sub_folder_path
    path_array.append(mp3_path)

In [14]:
songs=pd.DataFrame({
    "id":all_id,
    "genre":all_genre_top,
    "genres":all_genres,
    "subset":all_label_subset,
    'bit_rate': all_bit_rate,
    'path':path_array
})

# Filling in the missing Genre

In [ ]:
def get_biggest_genre(genres: list, genreData: pd.DataFrame) -> str:
    max_popularity = 0
    max_genre = ""
    for genre in genres:
        genre_popularity = genreData[genreData["genre_id"] == int(genre)]["#tracks"].values[0]
        genre_name = genreData[genreData["genre_id"] == int(genre)]["title"].values[0]
        if genre_popularity > max_popularity:
            max_popularity = genre_popularity
            max_genre = genre_name
    
    return max_genre

genreList = pd.read_csv("../../data/genres.csv")

wrong_songs=[]
for i, row in songs.iterrows():
    if pd.isna(row["genre"]):
        if(row["genres"] == "[]"):
            row["genre"] = "Unknown"
        else:
            row["genre"] = get_biggest_genre(
                [int(x) for x in row["genres"].strip('[]').split(',')]
                , genreList)
        wrong_songs.append(row)

print("Number of wrong songs:", len(wrong_songs))
print(pd.DataFrame(wrong_songs).value_counts("genre"))

# Clearing rows with no genre

In [ ]:
df_songs = songs.dropna(subset=["genre"])
print(df_songs.head())
print(df_songs["genre"].value_counts()) 

# Filtering and balancing the data

In [ ]:
fit_amount = 100

fitting_genres = [key for key, count in df_songs["genre"].value_counts().items() if count >= fit_amount]
print(fitting_genres)

In [10]:
df_songs = df_songs[df_songs["genre"].isin(fitting_genres)]

In [ ]:
print(df_songs["genre"].value_counts())

In [ ]:
# Group by genre and sample 2000 rows from each genre
df_songs_balanced = df_songs.groupby('genre').apply(lambda x: x.sample(fit_amount)).reset_index(drop=True)

print(df_songs_balanced["genre"].value_counts())

# Dropping unambigous genres

In [ ]:
df_songs_balanced = df_songs_balanced[~df_songs_balanced["genre"].isin(["Pop", "Experimental"])]
print(df_songs_balanced["genre"].value_counts())

In [ ]:
print(df_songs_balanced)
df_songs_balanced.to_csv("..\\..\\data\\tracks_1k_uncleaned.csv", index=False)

# Cutting large subset

In [104]:
df_songs_medium = df_songs[df_songs["subset"].isin(["small", "medium", "large"])]